In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
np.random.seed(1)
import sys
import sklearn
import sklearn.ensemble
%load_ext autoreload
%autoreload 2
from anchor import utils
import lime
import lime.lime_tabular
from int_met import *

C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
dataset_folder = 'Datasets/drugscom/'
train_raw = pd.read_csv(str(dataset_folder+'drugsComTrain_raw.csv'))
test_raw = pd.read_csv(str(dataset_folder+'drugsComTest_raw.csv'))

In [3]:
train = train_raw[['review','rating']]
test = train_raw[['review','rating']]
df_all = pd.concat([train,test]).reset_index()
del df_all['index']

In [4]:
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
from sklearn.model_selection import train_test_split

In [5]:
stops = set(stopwords.words('english'))
not_stop = ["aren't","couldn't","didn't","doesn't","don't","hadn't","hasn't","haven't","isn't","mightn't","mustn't","needn't","no","nor","not","shan't","shouldn't","wasn't","weren't","wouldn't"]
for i in not_stop:
    if i in stops:
        stops.remove(i)
    
stemmer = SnowballStemmer('english')
def review_to_words(raw_review):
    review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
    letters_only = re.sub('[^a-zA-Z]', ' ', review_text)
    words = letters_only.lower().split()
    meaningful_words = [w for w in words if not w in stops]
    stemming_words = [stemmer.stem(w) for w in meaningful_words]
    return( ' '.join(stemming_words))

In [6]:
%time df_all['review_clean'] = df_all['review'].apply(review_to_words)

Wall time: 4min 59s


In [7]:
df_all['sentiment'] = df_all["rating"].apply(lambda x: 1 if x > 5 else 0)

In [8]:
df_train, df_test = train_test_split(df_all, test_size=0.33, random_state=42) 

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

vectorizer = CountVectorizer(analyzer = 'word', 
                             tokenizer = None,
                             preprocessor = None, 
                             stop_words = None, 
                             min_df = 2,
                             max_features = 5000
                            )

In [10]:
%time X_train = vectorizer.fit_transform(df_train['review_clean'])
%time X_test = vectorizer.transform(df_test['review_clean'])

Wall time: 11.4 s
Wall time: 5.29 s


In [11]:
y_train = df_train['sentiment']
y_test = df_test['sentiment']

In [13]:
import pickle
pickling_on = open("textrf.pickle","rb")
c = pickle.load(pickling_on)
pickling_on.close()
predict_fn = lambda x: c.predict(x)

In [ ]:
c = sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=3, random_state=1)
c.fit(X_train, y_train)
predict_fn = lambda x: c.predict(x)
print('Train', sklearn.metrics.accuracy_score(y_train, predict_fn(X_train)))
print('Test', sklearn.metrics.accuracy_score(y_test, predict_fn(X_test)))

In [14]:
class_names = ['negative','positive']

In [15]:
from sklearn.pipeline import make_pipeline
pip_line = make_pipeline(vectorizer, c)

In [16]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [17]:
from anchor import anchor_text
explainer = anchor_text.AnchorText(nlp, ['sentiment'], use_unk_distribution=True)

In [18]:
def predict_lr(texts):
    return c.predict(vectorizer.transform(texts))

In [17]:
idx = 0
%time exp = explainer.explain_instance(df_test.review_clean.iloc[idx], predict_lr, threshold=0.95, use_proba=False)

Wall time: 41.9 s


In [18]:
idx = 0
%time exp = explainer.explain_instance(df_test.review_clean.iloc[idx], predict_lr, threshold=0.90, use_proba=False)

Wall time: 46.3 s


In [19]:
exp_fn_bulk = lambda x: np.array([explainer.explain_instance(x.iloc[idx], predict_lr, threshold=0.95, use_proba=False).names() for idx in range(len(x))])

In [ ]:
%time exp1 = exp_fn_bulk(df_test.review_clean[:5])

In [25]:
%time exp1 = exp_fn_bulk(df_test.review_clean[:5])

Wall time: 55.6 s


In [25]:
xx = df_test.review_clean[:401]

In [27]:
xx.drop_duplicates(inplace=True)

In [28]:
xx.shape

(400,)

In [ ]:
exp1 = exp_fn_bulk(xx[:400])
print('done')
exp2 = exp_fn_bulk(xx[:400])

In [46]:
i = calc_identity(exp1,exp2)
print(i)

(34.5, 262, 400)


In [49]:
s = calc_separability(exp2)
print (s)

(27794, 400, 160000, 17.37125)


In [50]:
a = np.array([str(i) for i in exp2])

In [51]:
vec = vectorizer.transform(a)

In [ ]:
np.array_equal(vec[1].indices,vec[3].indices)

In [52]:
v = vec.todense()

In [53]:
v.shape

(400, 5000)

In [54]:
sb = calc_stability(v, df_test.sentiment.iloc[:400].values)
print(sb)

C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\sklearn\cluster\k_means_.py:896: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


(72, 400)


In [33]:
import pickle
pickling_onn = open("exp1_400.pickle","rb")
exp1 = pickle.load(pickling_onn)
pickling_onn.close()

In [45]:
import pickle
pickling_on = open("exp1_400.pickle","wb")
pickle.dump(exp1, pickling_on)
pickling_on.close()